In [1]:
seed  = 1
from transformers import set_seed, BertTokenizer, TFAutoModelForMaskedLM, AutoTokenizer, get_scheduler, BertForPreTraining, BertForMaskedLM, BertConfig
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
import argparse, torch, datasets, ast, operator, gc, os
import torch.nn as nn
set_seed(seed)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description='search for best template according to dev set')

# parser.add_argument('--model', default='../Proxy_Prompt_LM/models/my_roberta/', type=str, help="pretrained model")
parser.add_argument('--dataset_dir', default='./datasets/', type=str, help="dataset dir")
parser.add_argument('--task', default='htl', type=str, help="task name")


args = parser.parse_args(args=[])
print(args)
print('''数据集'''+args.task)

Namespace(dataset_dir='./datasets/', task='htl')
数据集htl


In [7]:
if args.task =="eprstmt":
    few_shot_data_path = f'./data/k-shot/0/{args.task}/16-13'
    df_train = pd.read_csv(f'{few_shot_data_path}/test.csv',index_col=0,names=['text','labels'])
elif args.task == 'weibo':
    few_shot_data_path = f'{args.dataset_dir}weibo_senti_100k.csv'
    pd_all = pd.read_csv(few_shot_data_path,names=['labels','text'],header=0)
    pd_all = pd_all[
    pd_all["text"].apply(lambda x: len(x) >= 14 and len(x)<=15)
    ]
    
    labels = pd_all.labels.tolist()
    print(np.bincount(labels))
    min_label = min(np.bincount(labels))
    pd_pos = pd_all[pd_all.labels==1].sample(min_label)
    pd_neg = pd_all[pd_all.labels==0].sample(min_label)
    pd_output = pd.concat([pd_pos,pd_neg])
    pd_output = pd_output.sample(frac=1)

    print(pd_output)
    pd_output.to_csv(f'{args.dataset_dir}weibo_output.csv')


    
    # pd_dev_pos = pd_all[pd_all.labels==1].sample(250)
    # pd_dev_neg = pd_all[pd_all.labels==0].sample(250)   
    # pd_dev = pd.concat([pd_dev_pos,pd_dev_neg])
    # pd_dev = pd_dev.sample(frac=1)
    # pd_dev.to_csv(f'{args.dataset_dir}weibo_dev.csv')
    
    # pd_all = pd_all.drop(pd_dev.index)
    # print(len(pd_all))
    
    # pd_test_pos = pd_all[pd_all.labels==1].sample(250)
    # pd_test_neg = pd_all[pd_all.labels==0].sample(250)   
    # pd_test = pd.concat([pd_test_pos,pd_test_neg])
    # pd_test = pd_test.sample(frac=1)
    # pd_test.to_csv(f'{args.dataset_dir}weibo_test.csv')
    
    # pd_all = pd_all.drop(pd_test.index)
    # print(len(pd_all))
    
    # pd_all.to_csv(f'{args.dataset_dir}weibo_train.csv')
    
    # for seed in range(0,5):
    #     set_seed(seed)
    #     pd_train_pos = pd_all[pd_all.labels==1].sample(16)
    #     pd_train_neg = pd_all[pd_all.labels==0].sample(16)   
    #     pd_train = pd.concat([pd_train_pos,pd_train_neg])
    #     pd_train = pd_train.sample(frac=1)
    #     print(len(pd_train))
    #     pd_all.to_csv(f'{args.dataset_dir}weibo_split_{seed}.csv')
    
elif args.task =='waimai':
    few_shot_data_path = f'{args.dataset_dir}waimai_10k.csv'
    pd_all = pd.read_csv(few_shot_data_path,names=['labels','text'],header=0)
    pd_all = pd_all[
    pd_all["text"].apply(lambda x: len(x) >= 14 and len(x)<=15)
    ]
    labels = pd_all.labels.tolist()
    print(np.bincount(labels))
    min_label = min(np.bincount(labels))
    pd_pos = pd_all[pd_all.labels==1].sample(min_label)
    pd_neg = pd_all[pd_all.labels==0].sample(min_label)
    pd_output = pd.concat([pd_pos,pd_neg])
    pd_output = pd_output.sample(frac=1)

    print(pd_output)
    pd_output.to_csv(f'{args.dataset_dir}waimai_output.csv')
    
    # pd_dev_pos = pd_all[pd_all.labels==1].sample(250)
    # pd_dev_neg = pd_all[pd_all.labels==0].sample(250)   
    # pd_dev = pd.concat([pd_dev_pos,pd_dev_neg])
    # pd_dev = pd_dev.sample(frac=1)
    # pd_dev.to_csv(f'{args.dataset_dir}waimai_dev.csv')
    
    # pd_all = pd_all.drop(pd_dev.index)
    # print(len(pd_all))
    
    # pd_test_pos = pd_all[pd_all.labels==1].sample(250)
    # pd_test_neg = pd_all[pd_all.labels==0].sample(250)   
    # pd_test = pd.concat([pd_test_pos,pd_test_neg])
    # pd_test = pd_test.sample(frac=1)
    # pd_test.to_csv(f'{args.dataset_dir}waimai_test.csv')
    
    # pd_all = pd_all.drop(pd_test.index)
    # print(len(pd_all))
    
    # pd_all.to_csv(f'{args.dataset_dir}waimai_train.csv')
    
    # for seed in range(0,5):
    #     set_seed(seed)
    #     pd_train_pos = pd_all[pd_all.labels==1].sample(16)
    #     pd_train_neg = pd_all[pd_all.labels==0].sample(16)   
    #     pd_train = pd.concat([pd_train_pos,pd_train_neg])
    #     pd_train = pd_train.sample(frac=1)
    #     print(len(pd_train))
    #     pd_all.to_csv(f'{args.dataset_dir}waimai_split_{seed}.csv')
elif args.task=='ecommerce':
    few_shot_data_path = f'{args.dataset_dir}online_shopping_10_cats.csv'
    pd_all = pd.read_csv(few_shot_data_path,names=['cat','labels','text'],header=0,index_col=False)
    pd_all = pd_all[
    pd_all["text"].apply(lambda x: len(str(x)) >= 14 and len(str(x))<=15)
    ]
    labels = pd_all.labels.tolist()
    print(np.bincount(labels))
    min_label = min(np.bincount(labels))
    pd_pos = pd_all[pd_all.labels==1].sample(min_label)
    pd_neg = pd_all[pd_all.labels==0].sample(min_label)
    pd_output = pd.concat([pd_pos,pd_neg])
    pd_output = pd_output.sample(frac=1)

    print(pd_output)
    pd_output.to_csv(f'{args.dataset_dir}ecommerce_output.csv')
    
    # pd_dev_pos = pd_all[pd_all.labels==1].sample(1000)
    # pd_dev_neg = pd_all[pd_all.labels==0].sample(1000)   
    # pd_dev = pd.concat([pd_dev_pos,pd_dev_neg])
    # pd_dev = pd_dev.sample(frac=1)
    # pd_dev.to_csv(f'{args.dataset_dir}ecommerce_dev.csv')
    
    # pd_all = pd_all.drop(pd_dev.index)
    # print(len(pd_all))
    
    # pd_test_pos = pd_all[pd_all.labels==1].sample(2000)
    # pd_test_neg = pd_all[pd_all.labels==0].sample(2000)   
    # pd_test = pd.concat([pd_test_pos,pd_test_neg])
    # pd_test = pd_test.sample(frac=1)
    # pd_test.to_csv(f'{args.dataset_dir}ecommerce_test.csv')
    
    # pd_all = pd_all.drop(pd_test.index)
    # print(len(pd_all))
    
    # pd_all.to_csv(f'{args.dataset_dir}ecommerce_train.csv')
    
    # for seed in range(0,5):
    #     set_seed(seed)
    #     pd_train_pos = pd_all[pd_all.labels==1].sample(16)
    #     pd_train_neg = pd_all[pd_all.labels==0].sample(16)   
    #     pd_train = pd.concat([pd_train_pos,pd_train_neg])
    #     pd_train = pd_train.sample(frac=1)
    #     print(len(pd_train))
    #     pd_all.to_csv(f'{args.dataset_dir}ecommerce_split_{seed}.csv')
elif args.task=='douban':
    few_shot_data_path = f'{args.dataset_dir}douban_comments.csv'
    pd_all = pd.read_csv(few_shot_data_path,names=['movie_name','scores','text','labels'],header=0,index_col=0)
    print(len(pd_all))
    pd_all = pd_all[
    pd_all["text"].apply(lambda x: len(str(x)) >= 14 and len(str(x))<=15)
    ]
    labels = pd_all.labels.tolist()
    print(np.bincount(labels))
    min_label = min(np.bincount(labels))
    pd_pos = pd_all[pd_all.labels==1].sample(min_label)
    pd_neg = pd_all[pd_all.labels==0].sample(min_label)
    pd_output = pd.concat([pd_pos,pd_neg])
    pd_output = pd_output.sample(frac=1)

    print(pd_output)
    pd_output.to_csv(f'{args.dataset_dir}douban_output.csv')
    
    # pd_dev_pos = pd_all[pd_all.labels==1].sample(1000)
    # pd_dev_neg = pd_all[pd_all.labels==0].sample(1000)   
    # pd_dev = pd.concat([pd_dev_pos,pd_dev_neg])
    # pd_dev = pd_dev.sample(frac=1)
    # pd_dev.to_csv(f'{args.dataset_dir}douban_dev.csv')
    
    # pd_all = pd_all.drop(pd_dev.index)
    # print(len(pd_all))
    
    # pd_test_pos = pd_all[pd_all.labels==1].sample(2000)
    # pd_test_neg = pd_all[pd_all.labels==0].sample(2000)   
    # pd_test = pd.concat([pd_test_pos,pd_test_neg])
    # pd_test = pd_test.sample(frac=1)
    # pd_test.to_csv(f'{args.dataset_dir}douban_test.csv')
    
    # pd_all = pd_all.drop(pd_test.index)
    # print(len(pd_all))
    
    # pd_all.to_csv(f'{args.dataset_dir}douban_train.csv')
    
    # for seed in range(0,5):
    #     set_seed(seed)
    #     pd_train_pos = pd_all[pd_all.labels==1].sample(16)
    #     pd_train_neg = pd_all[pd_all.labels==0].sample(16)   
    #     pd_train = pd.concat([pd_train_pos,pd_train_neg])
    #     pd_train = pd_train.sample(frac=1)
    #     print(len(pd_train))
    #     pd_all.to_csv(f'{args.dataset_dir}douban_split_{seed}.csv')

elif args.task=='htl':
    few_shot_data_path = f'{args.dataset_dir}htl_7k.csv'
    pd_all = pd.read_csv(few_shot_data_path,names=['labels','text'],header=0,index_col=False)
    pd_all = pd_all[
    pd_all["text"].apply(lambda x: len(str(x)) >= 24 and len(str(x))<=25)
    ]
    labels = pd_all.labels.tolist()
    print(np.bincount(labels))
    min_label = min(np.bincount(labels))
    pd_pos = pd_all[pd_all.labels==1].sample(min_label)
    pd_neg = pd_all[pd_all.labels==0].sample(min_label)
    pd_output = pd.concat([pd_pos,pd_neg])
    pd_output = pd_output.sample(frac=1)

    pd_output.to_csv(f'{args.dataset_dir}htl_output.csv')

[ 27 128]


In [14]:
pd_all

,movie_name,scores,text,labels
1167,七宗罪,3.0,因为看的年代晚了所以惊喜全无～,0
1284,七宗罪,5.0,为什么拍出来十几年我才看到呢？,1
1340,七宗罪,5.0,结局真的很惊人。惨绝人寰……,1
1361,七宗罪,5.0,里面的布拉德皮特帅的掉渣……,1
1395,七月与安生,2.0,年龄大了，这种友谊想着都头疼。,0
...,...,...,...,...
50548,龙威父子,1.0,年轻不懂事的时候看过。。。。,0
50549,龙威父子,1.0,弱智版的断水流大师兄与何金银,0
50552,龙威父子,1.0,洪大这样的作品真是让人大跌眼镜,0
50567,龙威父子,2.0,原来黄晓明很早的时候就负责二了,0


In [24]:
douban = pd.read_csv('./datasets/douban_output.csv',names=['movie_name','scores','text','labels'],header=0,index_col=0)
del douban['movie_name'], douban['scores']
douban = douban[["labels", "text"]]
douban.to_csv(f'{args.dataset_dir}douban_output.csv')

In [31]:
ecommerce = pd.read_csv('./datasets/ecommerce_output.csv',names=['cat','labels','text'],header=0,index_col=0)
del ecommerce['cat']
ecommerce.to_csv(f'{args.dataset_dir}ecommerce_output.csv')
